In [0]:
dbutils.widgets.text("catalog", "usa", "catalog")
dbutils.widgets.text("schema", "movies", "schema")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
movie_titles_imdb = "https://datasets.imdbws.com/title.basics.tsv.gz"

In [0]:
volume_home_path = "/Volumes/{}}/{}}/temp".format(catalog, schema)

In [0]:
dbutils.fs.mkdirs(volume_home_path + "/imdb")

In [0]:
import requests

url = movie_titles_imdb
output_path = volume_home_path + "/imdb/title.basics.tsv.gz"

response = requests.get(url, stream=True)
with open(output_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

In [0]:
import gzip
import shutil

input_gz = volume_home_path + "/imdb/title.basics.tsv.gz"
output_tsv = volume_home_path + "/imdb/title.basics.tsv"

with gzip.open(input_gz, 'rb') as f_in, open(output_tsv, 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [0]:
df = spark.read.csv(output_tsv, sep='\t', header=True, inferSchema=True)


In [0]:
display(df)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("{}.{}.bronze_imdb_movies".format(catalog, schema))